 # Sample Codes for Linux Password Cracking in a Sample PenTest 

This is a DEMO to demostrate how to attack passwords on Linux using Python libraries in a Sample PenTest 

You are given

1- A shadow file (contains encrypted passwords in Linux)

2- An american-english dictionary file (contains all words in English)

# PennTest DEMO Scenario:

You are performing a penetration test in which you need to crack a password of a specific user. You
successfully downloaded the Shadow file from a remote server.

We overheard that Jane complained that her company mandates MONTHLY password change. She also bragged about how she is able to easily find new passwords and remember them perfectly. She said would start a password with a QUESTION MARK, followed by a short word she can remember. She will fill the rest of the password with the year and month. Based on what Jane said, we’d like to create the following assumptions.

The password is 8 characters long (Jane just wants to fulfill the minimum company requirement)

The first letter is “?” (Jane said it)

The last four letters are digits for the year (two digits) and month (digits). Today is in June 2020. 

So, the last first digits are “0620” or “2006”

The missing 3 characters will be a three-letter dictionary word with only one character capitalized.

Use the dictionary in “/usr/share/dict/american-english”. Extract all three-letter English words. You
need to remove the words with apostrophe such as “A’s”.

# Solution:

In [ ]:
#Extract usernames, salts, hashes from Shadow file

f=open("shadow","r") 
lines=f.read().split()
f.close()

usernames = []
salts =[]
hashes =[] 

for line in lines: 
	if "$6" in line:
		#print(line)
		sections = line.split("$")
		usernames.append(sections[0].split(":")[0])
		salts.append(sections[2])
		hashes.append(sections[3].split(":")[0])

result1 = dict(zip(usernames, hashes))      
        
print("Usernames and Hashes : " +  str(result1))

In [ ]:
#Create a password list for attack

usdict=open("american-english", "r")
usdictlines=usdict.read().split()
usdict.close()
#Extract all words which have 3 letters
WordList = []
for words in usdictlines:
    if len(words) ==3:
        WordList.append(words.capitalize())

from passlib.hash import sha512_crypt
from itertools import permutations

#The last four digits
dates= ["0620", "2006"]

#Creating combinations useing permutations
combos=[]
for n in WordList: 
	for d in dates:
		perm = permutations([n,"?",d],3)
		for p in list(perm):
			combos.append(''.join(p))
print("The length of all (combinations)permutations: ", len(combos))

#Cleaning combinations to get rid of unnecessary items. Here we remove items if they do not start with "?" and if they continue with date. It should continue 3 letters.
 
clear_combos =[]
for c in range (0,len(combos)):
    if combos[c][0] == "?" and combos[c][1] != "2" and combos[c][1] != "0" :
        clear_combos.append(combos[c])
print("The length of cleared (combinations)permutations: ", len(clear_combos))
print("Let's compare the combinations' hashes with the shadow passwords' hashes...")
print("As an example, our last 3 password in dictionary: ",clear_combos[-4:-1])

In [ ]:
#Crack the password by comparing real passwords' hashes with created list

for i in range(len(hashes)):
	for cc in range(len(clear_combos)):
		if hashes[i] == sha512_crypt.using(rounds=5000, salt=salts[i]).hash(clear_combos[cc]).split("$")[-1]:
			print("#####################")
			print("The password has been found by 'YOUR NAME' !!")
			print("Username is " + usernames[i])
			print("Password is " + clear_combos[cc])
			print("#####################")